# トークン化・変換・フィルタリング

In [1]:
import pandas as pd
import numpy as np

トークン化には spaCy を使います。
まずは spaCy を import しておきましょう

In [2]:
import spacy

2022-05-09 02:01:11.323480: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-09 02:01:11.323565: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## 形態素解析

spaCyを使うことで形態素解析とトークナイズを行うことができます。

In [3]:
nlp = spacy.load("ja_core_news_sm")
doc = nlp("今日は銀座から池袋へ向かいました。")
list(doc)

[今日, は, 銀座, から, 池袋, へ, 向かい, まし, た, 。]

## 見出し語（レンマ）化

`.lemma_` 属性を参照して見出し語化できます。

In [4]:
for token in doc:
    print(token.text, token.pos_, token.lemma_)

今日 NOUN 今日
は ADP は
銀座 PROPN 銀座
から ADP から
池袋 PROPN 池袋
へ ADP へ
向かい VERB 向かう
まし AUX ます
た AUX た
。 PUNCT 。


## 品詞によるフィルタリング

In [5]:
# 形態素の品詞の形態について書く (Universal Dependencies)
# https://hironsan.hatenablog.com/entry/visualize-dependencies-per-bunsetsu
doc = nlp("今日は銀座から池袋へ向かいました。")

# 内容語のみを残すようにフィルタリング
for token in nlp(doc):
    if token.pos_ in {"NOUN", "PROPN", "VERB"}:
        print(token.pos_, token.lemma_)

NOUN 今日
PROPN 銀座
PROPN 池袋
VERB 向かう


## ストップワードによるフィルタリング

ストップワードの集合を作成し、トークンがストップワードに含まれていれば出力しないようにします。
ストップワードを考慮してトークナイズする関数を定義してみましょう。

In [6]:
def tokenize(text, stopwords=set()):
    return [t.lemma_ for t in nlp(text) if t.lemma_ not in stopwords]

In [7]:
stopwords = {
    "は", "から", "へ",
    "ます", "た",
    "。", "、",
}

In [8]:
tokenize("今日は銀座から池袋へ向かいました。", stopwords=stopwords)

['今日', '銀座', '池袋', '向かう']

## 小文字化

strオブジェクトのlowerを使います。

In [9]:
doc = nlp("spaCyの練習")
for token in doc:
    print(token.text, token.text.lower())

spaCy spacy
の の
練習 練習


## 絵文字の変換

SNSのデータを処理する際には絵文字が出現することが多いです。
絵文字を事前に文字に変換しておくと処理しやすいです。

絵文字の変換には[emoji](https://github.com/carpedm20/emoji/)パッケージが便利です。

```{note}
emojiパッケージを利用するにはインストールが必要です。

    pip install emoji==1.7.0
```

In [10]:
import emoji

絵文字を文字列に変換します。

In [11]:
emoji.demojize("今日はケーキを食べます🍰")

'今日はケーキを食べます:shortcake:'

文字列を絵文字に変換します。

In [12]:
emoji.emojize("今日はケーキを食べます:shortcake:")

'今日はケーキを食べます🍰'

絵文字を除去します。

In [13]:
emoji.replace_emoji("今日はケーキを食べます🍰")

'今日はケーキを食べます'